In [1]:
import pandas as pd 
import numpy as np 

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
# Increase output limit
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Or right-click on output → "Create New View for Output"

In [4]:
import os, glob, pandas as pd

RAW_DIR = "data/"   # update to actual path
EXT = "csv"                # or "parquet"

files = sorted(glob.glob(os.path.join(RAW_DIR, f"*.{EXT}")))
def load_one(path):
    return pd.read_parquet(path) if path.endswith(".parquet") else pd.read_csv(path)

def variant_name(path):
    return os.path.splitext(os.path.basename(path))[0]

variants = {variant_name(p): load_one(p) for p in files}
list(variants.keys())


['Base', 'Variant I', 'Variant II', 'Variant III', 'Variant IV', 'Variant V']

In [5]:
import numpy as np

def infer_label_col(df):
    candidates = [c for c in df.columns if any(k in c.lower() for k in ["fraud","label","target","is_fraud"])]
    for c in candidates:
        u = sorted(df[c].dropna().unique())
        if len(u) <= 3 and set(u).issubset({0,1}):
            return c
    # Fallback: find any binary-int column
    for c in df.columns:
        u = sorted(df[c].dropna().unique())
        if len(u) <= 3 and set(u).issubset({0,1}):
            return c
    return None

def infer_time_col(df):
    # already datetime?
    dt_candidates = [c for c in df.columns if pd.api.types.is_datetime64_any_dtype(df[c])]
    if dt_candidates:
        return dt_candidates[0]
    # try parsing common names
    name_hits = [c for c in df.columns if any(k in c.lower() for k in ["time","date","timestamp","dt"])]
    for c in name_hits:
        try:
            parsed = pd.to_datetime(df[c], errors="raise", utc=True, infer_datetime_format=True)
            df[c] = parsed
            return c
        except Exception:
            continue
    return None

def candidate_groups(df, max_card=50):
    cats = []
    for c in df.columns:
        if df[c].dtype == "object" or str(df[c].dtype) == "category":
            if df[c].nunique(dropna=True) <= max_card:
                cats.append(c)
    return cats


In [6]:
def schema_report(name, df):
    label = infer_label_col(df)
    timec = infer_time_col(df)
    groups = candidate_groups(df)

    print(f"Variant: {name}")
    print("Rows, Cols:", df.shape)
    print("Label:", label)
    print("Time:", timec)
    print("Group candidates:", groups[:10])
    print("Null % (top 10):")
    print((df.isna().mean().sort_values(ascending=False).head(10) * 100).round(2))

for name, df in variants.items():
    schema_report(name, df)


C:\Users\Asus\AppData\Local\Temp\ipykernel_13344\2178368914.py:25: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="raise", utc=True, infer_datetime_format=True)


Variant: Base
Rows, Cols: (1000000, 32)
Label: fraud_bool
Time: date_of_birth_distinct_emails_4w
Group candidates: ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
Null % (top 10):
fraud_bool                   0.0
income                       0.0
device_fraud_count           0.0
device_distinct_emails_8w    0.0
keep_alive_session           0.0
device_os                    0.0
session_length_in_minutes    0.0
source                       0.0
foreign_request              0.0
proposed_credit_limit        0.0
dtype: float64


C:\Users\Asus\AppData\Local\Temp\ipykernel_13344\2178368914.py:25: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="raise", utc=True, infer_datetime_format=True)


Variant: Variant I
Rows, Cols: (1000000, 32)
Label: fraud_bool
Time: date_of_birth_distinct_emails_4w
Group candidates: ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
Null % (top 10):
fraud_bool                   0.0
income                       0.0
device_fraud_count           0.0
device_distinct_emails_8w    0.0
keep_alive_session           0.0
device_os                    0.0
session_length_in_minutes    0.0
source                       0.0
foreign_request              0.0
proposed_credit_limit        0.0
dtype: float64


C:\Users\Asus\AppData\Local\Temp\ipykernel_13344\2178368914.py:25: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="raise", utc=True, infer_datetime_format=True)


...

Variant: Variant III
Rows, Cols: (1000000, 34)
Label: fraud_bool
Time: date_of_birth_distinct_emails_4w
Group candidates: ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
Null % (top 10):
fraud_bool                   0.0
source                       0.0
phone_home_valid             0.0
phone_mobile_valid           0.0
bank_months_count            0.0
has_other_cards              0.0
proposed_credit_limit        0.0
foreign_request              0.0
session_length_in_minutes    0.0
income                       0.0
dtype: float64


C:\Users\Asus\AppData\Local\Temp\ipykernel_13344\2178368914.py:25: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="raise", utc=True, infer_datetime_format=True)


Variant: Variant IV
Rows, Cols: (1000000, 32)
Label: fraud_bool
Time: date_of_birth_distinct_emails_4w
Group candidates: ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
Null % (top 10):
fraud_bool                   0.0
income                       0.0
device_fraud_count           0.0
device_distinct_emails_8w    0.0
keep_alive_session           0.0
device_os                    0.0
session_length_in_minutes    0.0
source                       0.0
foreign_request              0.0
proposed_credit_limit        0.0
dtype: float64


C:\Users\Asus\AppData\Local\Temp\ipykernel_13344\2178368914.py:25: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(df[c], errors="raise", utc=True, infer_datetime_format=True)


Variant: Variant V
Rows, Cols: (1000000, 34)
Label: fraud_bool
Time: date_of_birth_distinct_emails_4w
Group candidates: ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
Null % (top 10):
fraud_bool                   0.0
source                       0.0
phone_home_valid             0.0
phone_mobile_valid           0.0
bank_months_count            0.0
has_other_cards              0.0
proposed_credit_limit        0.0
foreign_request              0.0
session_length_in_minutes    0.0
income                       0.0
dtype: float64


In [7]:
def prevalence_summary(df, label_col, time_col=None, freq="M"):
    pr = df[label_col].mean()
    out = {"overall_pos_rate": pr}
    if time_col:
        by_time = df.set_index(time_col).resample(freq)[label_col].mean().rename("pos_rate").to_frame()
        out["by_time"] = by_time
    return out

summaries = {}
for name, df in variants.items():
    ycol = infer_label_col(df)
    tcol = infer_time_col(df)
    summaries[name] = prevalence_summary(df, ycol, tcol)

# Example: inspect one
name0 = list(summaries.keys())[0]
summaries[name0]["overall_pos_rate"], summaries[name0].get("by_time", None)

(0.011029,
                                   pos_rate
 date_of_birth_distinct_emails_4w          
 1970-01-31 00:00:00+00:00         0.011029)

In [8]:
def time_checks(df, time_col):
    dft = df.dropna(subset=[time_col]).sort_values(time_col)
    return {
        "min_ts": dft[time_col].min(),
        "max_ts": dft[time_col].max(),
        "n_dupe_ts": dft.duplicated(subset=[time_col]).sum(),
        "rows_w_time": len(dft),
    }

for name, df in variants.items():
    tcol = infer_time_col(df)
    if tcol:
        print(name, time_checks(df, tcol))


Base {'min_ts': Timestamp('1970-01-01 00:00:00+0000', tz='UTC'), 'max_ts': Timestamp('1970-01-01 00:00:00.000000039+0000', tz='UTC'), 'n_dupe_ts': 999960, 'rows_w_time': 1000000}
Variant I {'min_ts': Timestamp('1970-01-01 00:00:00+0000', tz='UTC'), 'max_ts': Timestamp('1970-01-01 00:00:00.000000039+0000', tz='UTC'), 'n_dupe_ts': 999960, 'rows_w_time': 1000000}
Variant II {'min_ts': Timestamp('1970-01-01 00:00:00+0000', tz='UTC'), 'max_ts': Timestamp('1970-01-01 00:00:00.000000039+0000', tz='UTC'), 'n_dupe_ts': 999960, 'rows_w_time': 1000000}
Variant III {'min_ts': Timestamp('1970-01-01 00:00:00+0000', tz='UTC'), 'max_ts': Timestamp('1970-01-01 00:00:00.000000039+0000', tz='UTC'), 'n_dupe_ts': 999960, 'rows_w_time': 1000000}
Variant IV {'min_ts': Timestamp('1970-01-01 00:00:00+0000', tz='UTC'), 'max_ts': Timestamp('1970-01-01 00:00:00.000000039+0000', tz='UTC'), 'n_dupe_ts': 999960, 'rows_w_time': 1000000}
Variant V {'min_ts': Timestamp('1970-01-01 00:00:00+0000', tz='UTC'), 'max_ts': T

In [9]:
def group_prevalence(df, label_col, group_col, min_size=200):
    grp = df.groupby(group_col)[label_col].agg(["count","mean"]).sort_values("count", ascending=False)
    return grp[grp["count"] >= min_size]

for name, df in variants.items():
    ycol = infer_label_col(df)
    gcols = candidate_groups(df)
    print(f"\n{name} group prevalence (top 2 candidates):")
    for gc in gcols[:2]:
        print(gc)
        print(group_prevalence(df, ycol, gc).head(10))



Base group prevalence (top 2 candidates):
payment_type
               count      mean
payment_type                  
AB            370554  0.011251
AA            258249  0.005282
AC            252071  0.016698
AD            118837  0.010822
AE               289  0.003460
employment_status
                    count      mean
employment_status                  
CA                 730252  0.012186
CB                 138288  0.006891
CF                  44034  0.001930
CC                  37758  0.024684
CD                  26522  0.003770
CE                  22693  0.002336
CG                    453  0.015453

Variant I group prevalence (top 2 candidates):
payment_type
               count      mean
payment_type                  
AB            364724  0.011060
AA            260651  0.005417
AC            253419  0.016976
AD            120964  0.010582
AE               242  0.004132
employment_status
                    count      mean
employment_status                  
CA               

In [10]:
PROC_DIR = "data/baf_processed"
os.makedirs(PROC_DIR, exist_ok=True)

def to_processed(df, label_col, time_col, drop_cols=None):
    dfx = df.copy()
    if time_col and not pd.api.types.is_datetime64_any_dtype(dfx[time_col]):
        dfx[time_col] = pd.to_datetime(dfx[time_col], errors="coerce", utc=True, infer_datetime_format=True)
    dfx[label_col] = dfx[label_col].astype("int8")
    # Drop obvious IDs/leakage if known
    if drop_cols:
        dfx = dfx.drop(columns=[c for c in drop_cols if c in dfx.columns])
    # Cast object low-cardinality to category for memory efficiency
    for c in dfx.columns:
        if dfx[c].dtype == "object" and dfx[c].nunique(dropna=True) <= 1000:
            dfx[c] = dfx[c].astype("category")
    return dfx

for name, df in variants.items():
    ycol = infer_label_col(df)
    tcol = infer_time_col(df)
    dfx = to_processed(df, ycol, tcol, drop_cols=None)  # provide known IDs if any
    ext = "parquet"
    out = os.path.join(PROC_DIR, f"{name}.parquet")
    dfx.to_parquet(out, index=False)
    print("Wrote:", out, dfx.shape)


ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.